In [2]:
from scene import Scene
import taichi as ti
from taichi.math import *
import numpy as np

ti.init(arch=ti.gpu, debug=True)

[Taichi] Starting on arch=cuda


In [4]:
x = ti.field(float, shape=(3, 3))
a = np.arange(9).reshape(3, 3).astype(np.int32)
x.from_numpy(a)
print(x)
type(x)

[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]


taichi.lang.field.ScalarField

In [5]:
arr = x.to_numpy()
print(arr)
type(arr)

[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]


numpy.ndarray

In [6]:
field = ti.Vector.field(3, int, shape=(256, 512))
field.shape  # (256, 512)
field.n      # 3

array = field.to_numpy()
array.shape  # (256, 512, 3)
print(type(array))

field.from_numpy(array)  # the input array must in the shape (256, 512, 3)
print(type(field))

<class 'numpy.ndarray'>
<class 'taichi.lang.matrix.MatrixField'>


## Render